In [5]:
from wand.image import Image as wand_image
from PIL import Image as pil_image
from glob import glob
import os
from pathlib import Path
import traceback
import numpy as np

from bounded_pool_executor import BoundedProcessPoolExecutor
import multiprocessing as mp

In [6]:
input_path = 'mydog/*'
output_dir = 'mydog_cropped'

Path(output_dir).mkdir(parents=True, exist_ok=True)

file_list = glob(input_path)
file_list

['mydog/alvan-nee-9M0tSjb-cpA-unsplash.jpeg',
 'mydog/alvan-nee-brFsZ7qszSY-unsplash.jpeg',
 'mydog/alvan-nee-Id1DBHv4fbg-unsplash.jpeg',
 'mydog/alvan-nee-bQaAJCbNq3g-unsplash.jpeg',
 'mydog/color-theme.psd',
 'mydog/alvan-nee-eoqnr8ikwFE-unsplash.jpeg']

In [7]:
def img_resize(file, size=512):
    with pil_image.open(file) as im:
        width, height = im.size
        if width < height:
            newWidth = size
            newHeight = int(height / (width / size))
            cropTop = (newHeight - size) // 2
            cropBottom = cropTop + size
            crop = (0, cropTop, size, cropBottom)
        else:
            newHeight = size
            newWidth = int(width / (height / size))
            cropLeft = (newWidth - size) // 2
            cropRight = cropLeft + size
            crop = (cropLeft, 0, cropRight, size)
        imResize = im.resize((newWidth, newHeight))
        imCrop = imResize.crop(crop)
        basename = file.split('/')[-1].split('.')[0]
        
        imCrop.save(os.path.join(output_dir, "{}_cropped.jpeg".format(basename)))

In [8]:
def psd_resize(file, size=512):
    myImage = wand_image(filename="{}[0]".format(file))
    myImage.format = "png"

    width, height = myImage.size
    if width < height:
        newWidth = size
        newHeight = int(height / (width / size))
    else:
        newHeight = size
        newWidth = int(width / (height / size))

    myImage.resize(newWidth, newHeight)
    # print(newWidth, newHeight)
    myImage.crop(width=size, height=size, gravity="center")
    basename = file.split('/')[-1].split('.')[0]
    
    myImage.save(filename=os.path.join(output_dir, "{}_cropped.png".format(basename)))

In [9]:
def handler(file_list):
    for f in file_list:
        try:
            postfix = f.split('.')[-1]
            if postfix == 'jpeg' or postfix == 'jpg' or postfix == 'png':
                img_resize(f)
            elif postfix == 'psd':
                psd_resize(f)
        except Exception:
            print(postfix, f)
            print(traceback.format_exc())

In [10]:
def batch_run():
    _PARALLEL = mp.cpu_count()
    _LEN = len(file_list)
    if _PARALLEL > _LEN:
        _PARALLEL = _LEN

    file_list_split = np.array_split(file_list, _PARALLEL)

    with BoundedProcessPoolExecutor(max_workers=(_PARALLEL)) as executor:
        _ = executor.map(handler, file_list_split)

In [11]:
batch_run()